A MACHINE LEARNING MODEL BUILT TO PREDICT STUDENT'S ACADEMIC PEROFRMANCE GIVEN THEIR INDIVIDUAL FEATURES

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings 
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('data/StudentsPerformance 2.csv')

In [4]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [11]:
df.describe()

,math score,reading score,writing score
count,1000.00000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
25%,57.00000,59.000000,57.750000
50%,66.00000,70.000000,69.000000
75%,77.00000,79.000000,79.000000
max,100.00000,100.000000,100.000000


In [12]:
df.isna().sum()

gender                         0
race/ethnicity                 0
parental level of education    0
lunch                          0
test preparation course        0
math score                     0
reading score                  0
writing score                  0
dtype: int64

In [14]:
df.duplicated().sum()

0

In [15]:
df.nunique()

gender                          2
race/ethnicity                  5
parental level of education     6
lunch                           2
test preparation course         2
math score                     81
reading score                  72
writing score                  77
dtype: int64

Create total score and avarage score for each students

In [17]:
df['total_score'] = df['math score']+df['reading score']+df['writing score']
df['avg_score'] = df['total_score']/3
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,total_score,avg_score
0,female,group B,bachelor's degree,standard,none,72,72,74,218,72.666667
1,female,group C,some college,standard,completed,69,90,88,247,82.333333
2,female,group B,master's degree,standard,none,90,95,93,278,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,148,49.333333
4,male,group C,some college,standard,none,76,78,75,229,76.333333


In [19]:
X = df.drop(columns=['math score'],axis=1)
y = df['math score']

In [22]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
  
  [
    ("OneHotEncoder", oh_transformer, cat_features),
    ("StandardScaler", num_transformer, num_features),
  ]

)


In [23]:
X = preprocessor.fit_transform(X)

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

X_train.shape, X_test.shape

((800, 21), (200, 21))

In [25]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_model(true, predicted):
  mae = mean_absolute_error(true, predicted)
  mse = mean_squared_error(true, predicted)
  r2_square = r2_score(true, predicted)
  return mae, mse, r2_square

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor


models = {
    "Linear Regression": LinearRegression(),
    "Lasso": linear_model.Lasso(),
    "Decision Tree": DecisionTreeRegressor(),
    "K Neighbors": KNeighborsRegressor()

}

In [30]:
model_list = []
r2_list = []

for i in range(len(list(models))):
  model = list(models.values())[i]
  model.fit(X_train, y_train)

  y_train_pred = model.predict(X_train)
  y_test_pred = model.predict(X_test)

  model_train_mae, model_train_mse, model_train_r2_square = evaluate_model(y_train, y_train_pred)
  model_test_mae, model_test_mse, model_test_r2_square = evaluate_model(y_test, y_test_pred)

  print(list(models.keys())[i])
  model_list.append(list(models.keys())[i])

  print('model performance for training set')
  print("MAE: {:.4f}".format(model_train_mae))
  print("MSE: {:.4f}".format(model_train_mse))
  print("r2_square: {:.4f}".format(model_train_r2_square))
  
  print('model performance for test set')
  print("MAE: {:.4f}".format(model_test_mae))
  print("MSE: {:.4f}".format(model_test_mse))
  print("r2_square: {:.4f}".format(model_test_r2_square))
  r2_list.append(model_test_r2_square)

  print('='*35)
  print('\n')


  


Linear Regression
model performance for training set
MAE: 0.0000
MSE: 0.0000
r2_square: 1.0000
model performance for test set
MAE: 0.0000
MSE: 0.0000
r2_square: 1.0000


Lasso
model performance for training set
MAE: 3.7439
MSE: 22.4492
r2_square: 0.9004
model performance for test set
MAE: 3.7579
MSE: 22.2508
r2_square: 0.9086


Decision Tree
model performance for training set
MAE: 0.0000
MSE: 0.0000
r2_square: 1.0000
model performance for test set
MAE: 3.0700
MSE: 14.8700
r2_square: 0.9389


K Neighbors
model performance for training set
MAE: 3.3838
MSE: 18.3105
r2_square: 0.9188
model performance for test set
MAE: 4.0460
MSE: 26.8204
r2_square: 0.8898




In [32]:
pd.DataFrame(list(zip(model_list,r2_list)), columns=['Model name', 'R2_score']).sort_values(by=['R2_score'], ascending=False)

,Model name,R2_score
0,Linear Regression,1.000000
2,Decision Tree,0.938892
1,Lasso,0.908560
3,K Neighbors,0.889781
